<a href="https://colab.research.google.com/github/DurhamARC/raga-pose-estimation/blob/jo-branch/RagaPoseEstimationColab.ipynb"
target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pose Detection with OpenPose

This notebook uses an open source project [CMU-Perceptual-Computing-Lab/openpose](https://github.com/CMU-Perceptual-Computing-Lab/openpose.git) to detect/track multi person poses on a given video.

Please read the [OpenPose license](https://github.com/CMU-Perceptual-Computing-Lab/openpose/blob/master/LICENSE) before running this script.


##Install OpenPose

In [5]:
'''#@title
!pip install ffmpeg-python
import os
from os.path import exists, join, basename, splitext

def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii'))) 


git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
!rm -rf openpose
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.17/cmake-3.17.2-Linux-x86_64.tar.gz
  !tar xfz cmake-3.17.2-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  # --recursive necessary in the line below, as otherwise you can (sometimes) get "lpthreads" errors in cmake ("undefined reference to `pthread_create'" etc). See, for example, https://github.com/facebookarchive/caffe2/issues/1234
  !sed -i 's/execute_process(COMMAND git checkout --recursive master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  !cd openpose && git submodule update --init --recursive --remote
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # build openpose
  # CUDA
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc` 
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc` 
  # CPU 
  #!cd openpose && rm -rf build || true && mkdir build && cd build && cmake -DGPU_MODE=CPU_ONLY -DUSE_MKL=OFF .. && make -j`nproc`'''

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 13)

Workaround for server problem (run if you see `file DOWNLOAD HASH mismatch` in the previous output):

In [6]:
#@title
# Workaround for server connection problem ("file DOWNLOAD HASH mismatch"),see, e.g., 
# https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/1602#issuecomment-641653411

!apt-get install unzip

!wget -O models.zip  --no-check-certificate -r 'https://drive.google.com/uc?id=1QCSxJZpnWvM00hx49CJ2zky7PWGzpcEh&export=download'
!unzip -o models.zip -d openpose

zsh:1: command not found: apt-get
will be placed in the single file you specified.

--2022-11-23 12:58:16--  https://drive.google.com/uc?id=1QCSxJZpnWvM00hx49CJ2zky7PWGzpcEh&export=download
Resolving drive.google.com (drive.google.com)... 142.250.178.14
Connecting to drive.google.com (drive.google.com)|142.250.178.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘models.zip’

models.zip              [ <=>                ]   2.21K  --.-KB/s    in 0s      

2022-11-23 12:58:16 (25.4 MB/s) - ‘models.zip’ saved [2261]

Loading robots.txt; please ignore errors.
--2022-11-23 12:58:16--  https://drive.google.com/robots.txt
Reusing existing connection to drive.google.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: ‘models.zip’

models.zip              [ <=>                ]     570  --.-KB/s    in 0s      

2022-11-23 12:58:16 (36.2 MB/s) - ‘models.zip’ saved [570]

--2022-11-23 1

## Processing
Optionally run OpenPose, then do post-processing which can include visualisations.



### Preparation

Check out the raga-pose-estimation module from github.

In [7]:
!git clone https://github.com/DurhamARC/raga-pose-estimation.git raga-pose-estimation

branch = input('Enter the branch to check out (defaults to jo-branch): ') or 'jo-branch'
!cd raga-pose-estimation && git checkout $branch

fatal: destination path 'raga-pose-estimation' already exists and is not an empty directory.
Already on 'master'
Your branch is up to date with 'origin/master'.


### Upload files

If your video or JSON files are not available via a public URL, use the Files menu on the left to upload them. You can then copy their path by hovering, clicking on the three dots, and selecting **Copy path**.

### Process

The next cell generates a form which you can use to run OpenPose on a video followed by post-processing, or just do the post-processing on a directory of JSON files produced by OpenPose.

#### Options

| Option  |  Description |
|---|---|
| Output directory | Path to the directory in which to output CSV |
| OpenPose directory | Path to the directory in which openpose is installed. |
| OpenPose arguments | Additional arguments to pass to OpenPose. See the [OpenPose flags](https://github.com/CMU-Perceptual-Computing-Lab/openpose/blob/master/include/openpose/flags.hpp) for details. |
| Input video | Path to the video file on which to run openpose.<br>Can be a remote URL, e.g. from the [Dropbox folder](https://www.dropbox.com/sh/fcbe7ebrvuutfgh/AACAe78sSZRMjFhJQAq1wM-Ra?dl=0).<br>To use a specific video, navigate to the respective subfolder on Dropbox, right-click on the video and choose "copy link location". Then paste in the field below. |
| Crop video | Whether to crop the video before processing. |
| Crop width<br>Crop height<br>Crop top left x<br>Crop top left y | Parameters for cropping the video before processing.<br>(x,y) are the coordinates of the top-left of the cropped rectangle, as measured in pixels from the top-left corner.<br>You can click **Preview Crop** to preview the crop rectangle on the first frame of the video. |
| Input JSON | Path to a directory of previously generated openpose JSON files.<br>Can be a remote URL to a zipped file. |
| Number of people | Number of people to include in output |
| Create model video | Whether to create a video showing the poses on a blank background |
| Create overlay video | Whether to create a video showing the poses as an overlay |
| Download results | Whether to download the zipped results |
| width | Width of original video (mandatory for creating video if  not providing input-video) |
| height | Height of original video (mandatory for creating video if  not providing input-video) |
| Confidence threshold | Items with a confidence lower than the threshold will be replaced by values from a previous frame. |
| Smoothing parameters | If set, applies a smoother to the OpenPose output. <br>(See [README](README.md) for more details.) |
| Group of body parts to analyse | Group of body parts to include in output.<br>If "Choose..." is selected you will be able to select the individual parts in *Body parts to analyse* |
| Body parts to analyse | Body parts to include in output. Use `Ctrl` or `Cmd` to select multiple. |
| Flatten |Export CSV in flattened format, i.e. with a single header row.<br>(See [README](README.md) for more details.) |

Run the first cell to generate a form. Once you have filled in the form, click "Generate parameters", then run the next cell.


In [8]:
#@title
from datetime import datetime
import os
import sys
import tempfile
import urllib
import urllib.request

import cv2
from google.colab import files, output
from google.colab.patches import cv2_imshow
import ipywidgets as widgets

sys.path.append(os.path.join(os.getcwd(), 'raga-pose-estimation'))

import run_pose_estimation

from raga_pose_estimation.openpose_parts import (
    OpenPosePartGroups,
    OpenPoseParts,
)

def default_dirname(prefix='output'):
    dirname = datetime.now().strftime('%Y-%m-%d_%H%M%S')
    return f'{prefix}/{dirname}'

output_dir = default_dirname()

style = {'description_width': '200px'}
layout = {'width': '400px'}

items = {
    'output_dir': widgets.Text(
        value=output_dir,
        description='Output directory',
        style=style, 
        layout=layout
    ),
    'openpose_dir': widgets.Text(
        value='openpose',
        description='OpenPose directory',
        style=style, 
        layout=layout
    ),
    'openpose_args': widgets.Text(
        value='',
        description='Additional OpenPose arguments',
        style=style, 
        layout=layout
    ),
    'input_video': widgets.Text(
        value='',
        description='Input video',
        style=style, 
        layout=layout
    ),
    'crop_video': widgets.Checkbox(
        value=False,
        description='Crop video',
        style=style,
        layout=layout
    ),
    'crop_video_w': widgets.BoundedIntText(
        value=None,
        description='Crop width',
        style=style, 
        layout=layout,
        disabled=True,
        min=1,
        max=100000
    ),
    'crop_video_h': widgets.BoundedIntText(
        value=None,
        description='Crop height',
        style=style, 
        layout=layout,
        disabled=True,
        min=1,
        max=100000
    ),
    'crop_video_x': widgets.BoundedIntText(
        value=None,
        description='Crop top left x',
        style=style, 
        layout=layout,
        disabled=True,
        min=0,
        max=100000
    ),
    'crop_video_y': widgets.BoundedIntText(
        value=None,
        description='Crop top left y',
        style=style, 
        layout=layout,
        disabled=True,
        min=0,
        max=100000
    ),
    'crop_preview': widgets.Button(
        value=None,
        description='Preview crop',
        style=style, 
        layout=layout
    ),
    'input_json': widgets.Text(
        value='',
        description='Input JSON',
        style=style, 
        layout=layout
    ),
    'number_of_people': widgets.IntSlider(
        value=1,
        min=1,
        max=10,
        step=1,
        description='Number of people',
        style=style, 
        layout=layout
    ),
    'performer_names': widgets.Text(
        value=["April", "Ben", "Charlie"],
        description='Performer Names',
        style=style, 
        layout=layout
    ),
    'create_model_video': widgets.Checkbox(
        value=False,
        description='Create model video',
        style=style, 
        layout=layout
    ),
    'create_overlay_video': widgets.Checkbox(
        value=False,
        description='Create overlay video',
        style=style, 
        layout=layout
    ),
    'download_results': widgets.Checkbox(
        value=False,
        description='Download results',
        style=style, 
        layout=layout
    ),
    'width': widgets.IntText(
        value=0,
        description='Width of video',
        style=style
    ),
    'height': widgets.IntText(
        value=0,
        description='Height of video',
        style=style
    ),
    'confidence_threshold': widgets.FloatSlider(
        value=0.7,
        min=0,
        max=1.0,
        step=0.05,
        description='Confidence Threshold',
        style=style, 
        layout=layout
    ),
    'apply_smoother': widgets.Checkbox(
        value=True,
        description='Apply smoother',
        style=style, 
        layout=layout
    ),
    'smoothing_window': widgets.IntSlider(
        value=21,
        min=1,
        max=51,
        step=2,
        description='Smoothing window',
        style=style, 
        layout=layout
    ),
    'smoothing_polyorder': widgets.IntSlider(
        value=2,
        min=1,
        max=12,
        step=1,
        description='Smoothing polynomial order',
        style=style, 
        layout=layout
    ),
    'body_part_group': widgets.RadioButtons(
        options=[
            ('All', list(OpenPoseParts)),
            ('Upper', OpenPosePartGroups.UPPER_BODY_PARTS),
            ('Lower', OpenPosePartGroups.LOWER_BODY_PARTS),
            ('Choose...', [])
        ],
        value=list(OpenPoseParts),
        description='Group of body parts to analyse',
        disabled=False,
        style=style, 
        layout=layout
    ),
    'body_parts': widgets.SelectMultiple(
        options=[(x.value, x) for x in list(OpenPoseParts)],
        value=list(OpenPoseParts),
        description='Body parts to analyse',
        disabled=True,
        style=style, 
        layout=layout
    ),
    'flatten': widgets.Checkbox(
        value=False,
        description='Flatten CSV output',
        style=style, 
        layout=layout
    ),
    'run': widgets.Button(
        description='Generate parameters',
        disabled=False,
        style=style,
        layout=layout,
        tooltip='Generate parameters for openpose or post-processing',
        icon='check'
    ),
    'trial_no': widgets.Text(
        value='',
        description='Trial Number',
        style=style, 
        layout=layout
    ),
    'output': widgets.Output(layout={'border': '1px solid gray'})
}

def update_body_parts(*args):
    if items['body_part_group'].value == []:
        items['body_parts'].disabled = False
    else:
        items['body_parts'].disabled = True
        items['body_parts'].value = items['body_part_group'].value

def update_smoother(*args):
    if items['apply_smoother'].value == True:
        items['smoothing_window'].disabled = False
        items['smoothing_polyorder'].disabled = False
    else:
        items['smoothing_window'].disabled = True
        items['smoothing_polyorder'].disabled = True

def get_input_path(path_or_url, compressed=False):
    if path_or_url is None:
        return None
    
    parsed = urllib.parse.urlparse(path_or_url)
    input_dir = default_dirname('input')

    if parsed.scheme == "":
        filepath = path_or_url
    else:
        print(parsed.query)
        if parsed.query == 'dl=0':
            # Dropbox URL - change to dl=1 to download directly
            replaced = parsed._replace(query='dl=1')
            path_or_url = urllib.parse.urlunparse(replaced)

        print(f"Retrieving file from {path_or_url}")
        filename = parsed.path.rsplit('/', 1)[1]
        os.makedirs(input_dir, exist_ok=True)
        filepath = f"{input_dir}/{filename}"
        urllib.request.urlretrieve(path_or_url, filepath)

    if compressed:
        os.makedirs(input_dir, exist_ok=True) 
        with file(filepath,'r') as f:
          if zipfile.is_zipfile(f):
              zip = zipfile.ZipFile(f)
              zip.extractall(input_dir)
              filepath = input_dir

    return filepath
  
def update_crop_params(*args):
    if items['crop_video'].value == True:
        items['crop_video_w'].disabled = False
        items['crop_video_h'].disabled = False
        items['crop_video_x'].disabled = False
        items['crop_video_y'].disabled = False
        items['crop_preview'].disabled = False
    else:
        items['crop_video_w'].disabled = True
        items['crop_video_h'].disabled = True
        items['crop_video_x'].disabled = True
        items['crop_video_y'].disabled = True
        items['crop_preview'].disabled = True

def preview_crop(*args):
  output.clear()
  display(ui)
  input_video = get_input_path(items['input_video'].value or None)
  crop_video = items['crop_video'].value
  w = items['crop_video_w'].value
  h = items['crop_video_h'].value
  x = items['crop_video_x'].value
  y = items['crop_video_y'].value
    
  if input_video and crop_video:
    cap = cv2.VideoCapture(input_video)
    ret, frame = cap.read()
    cap.release()
    frame = cv2.rectangle(frame,(x, y),(x+w,y+h),(0,255,0),2)
    cv2_imshow(frame)


run_pose_estimation_params = {}

def on_button_clicked(b):
    global run_pose_estimation_params
    items['output'].clear_output()
    with items['output']:
        try:
            if not items['output_dir'].value:
                print("You must select an output directory.")
            else:
                input_video = get_input_path(items['input_video'].value or None)
                input_json = get_input_path(items['input_json'].value or None)
                smoothing_parameters = None
                if items['apply_smoother'].value == True:
                  smoothing_parameters = (items['smoothing_window'].value,
                                          items['smoothing_polyorder'].value)
                
                crop_rectangle = None
                if items['crop_video'].value:
                  crop_rectangle = (
                      items['crop_video_w'].value,
                      items['crop_video_h'].value,
                      items['crop_video_x'].value,
                      items['crop_video_y'].value
                  )

                run_pose_estimation_params = {
                    'output_dir': items['output_dir'].value,
                    'openpose_dir': items['openpose_dir'].value or None,
                    'openpose_args': items['openpose_args'].value or None,
                    'input_video': input_video,
                    'input_json': input_json,
                    'crop_rectangle': crop_rectangle,
                    'number_of_people': items['number_of_people'].value,
                    'create_model_video': items['create_model_video'].value,
                    'create_overlay_video': items['create_overlay_video'].value,
                    'width': items['width'].value,
                    'height': items['height'].value,
                    'confidence_threshold': items['confidence_threshold'].value,
                    'smoothing_parameters': smoothing_parameters,
                    'body_parts': items['body_parts'].value,
                    'flatten': items['flatten'].value,
                    'trial_no': items['trial_no'].value,
                    'performer_names': items['performer_names'].value
                }
                print("Parameters saved. Run the next cell to execute run_pose_estimation.")
        except SystemExit:
            print('Aborted.')
    
items['body_part_group'].observe(update_body_parts, 'value')
items['apply_smoother'].observe(update_smoother, 'value')
items['crop_video'].observe(update_crop_params, 'value')
items['crop_preview'].on_click(preview_crop)
items['run'].on_click(on_button_clicked)

ui = widgets.VBox(list(items.values()))
display(ui)


/Users/joannesheppard/Documents/raga-pose-estimation/ragaenv/lib/python3.7/site-packages/google/colab/data_table.py:30: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  from IPython.utils import traitlets as _traitlets


Run the next cell to run openpose/post-processing.

In [ ]:
#@title
print("Running openpose/post-processing...")
try:
  # Run openpose
  run_pose_estimation.run_pose_estimation(
      **run_pose_estimation_params
  )
except Exception as e:
  print('OpenPose Checkpoint')

try:
  output_dir = items['output_dir'].value

except Exception as e:
  print('Output Directory Checkpoint')

try:
  # Download outputs
  zip_file = f"{output_dir}.zip"
  !zip -r $zip_file $output_dir
  print(f'Created zip archive at {zip_file}')

  if(items['download_results'].value):
      files.download(zip_file)

  # Update output dir if using the default
  new_output_dir = default_dirname()
  if items['output_dir'].value.startswith(new_output_dir[:11]):
      items['output_dir'].value = new_output_dir

except Exception as e:
  print('Zip Checkpoint')

'''except Exception as e:
  print('Aborted!')
  print('Please check the parameters in the form and re-run "Generate Parameters".')'''


### Download output

Use the Files menu on the left to find the folder you'd like to download, then use the three dots button and click **Copy path**. Run the next cell, and paste the path in the input box. The cell will create a zip file which you can download from the Files menu.

In [16]:
#@title
dirname = input('Path to download: ')
zip_file = f"{dirname}.zip"

!zip -r $zip_file $dirname 

print(f"Created file {zip_file}. Use the files menu on the left to download it. You may need to refresh the files first.")

  adding: Users/joannesheppard/Documents/raga-pose-estimation/example_files/example_1person/short_video.mp4 (deflated 0%)
Created file /Users/joannesheppard/Documents/raga-pose-estimation/example_files/example_1person/short_video.mp4.zip. Use the files menu on the left to download it. You may need to refresh the files first.


Alternative way of downloading results:

In [17]:
'''#@title
from datetime import datetime

from google.colab import drive

drive.mount('/content/drive')

output_dir = "output"
zip_file = f"{output_dir}.zip"
!zip -r $zip_file $output_dir
!mv '/content/output.zip' "/content/drive/My Drive/`date +%Y-%m-%d_%H%M%S`.zip"

!echo "Copy of output folder moved to Drive"'''


KeyError: 'CLOUDSDK_CONFIG'

### View videos
Convert the avis to mp4s so they can be seen here.

In [18]:
!ffmpeg -y -loglevel info -i $output_dir/video_blank.avi $output_dir/video_blank.mp4
!ffmpeg -y -loglevel info -i $output_dir/video_overlay.avi $output_dir/video_overlay.mp4

ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.102)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/5.1.2 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable-indev=jack --enable-videotoolb

Display the visualisation as a model on a blank background:

In [ ]:
show_local_mp4_video(f'{output_dir}/video_blank.mp4', width=768, height=576)

Next display the visualisation as an overlay:

In [19]:
show_local_mp4_video(f'{output_dir}/video_overlay.mp4', width=768, height=576)

FileNotFoundError: [Errno 2] No such file or directory: 'output/2022-11-16_150727/video_overlay.mp4'

### View CSV


In [20]:
#@title
import pandas as pd

dir = input('CSV file: ')
flatten = input('CSV was flattened? (y/n):')
if flatten.lower() == 'y':
  df = pd.read_csv(filepath)
else:
  df = pd.read_csv(filepath, header=[0,1], index_col=0)

df

NameError: name 'filepath' is not defined

##Tests

### Detect poses on a test video

Download video from Dropbox

The Link to the Dropbox folder is https://www.dropbox.com/sh/fcbe7ebrvuutfgh/AACAe78sSZRMjFhJQAq1wM-Ra?dl=0 . To use a specific video, navigate to the respective subfolder on Dropbox, right-click on the video and choose "copy link location". Then paste in the "videolink" field below.  



In [ ]:
#@title Paste link to video here { display-mode: "form" }
videolink="https://www.dropbox.com/sh/fcbe7ebrvuutfgh/AABIpUxn2jSq_el0OpdDifXha/Apoorva_experiment_clips/Apoorva_raga_8_muted.mp4?dl=0"#@param {type:"string"}

In [ ]:
#@title
!rm -rf full_video.mp4

!wget -O full_video.mp4 $videolink 

Access the video, cut the first 5 seconds and do the pose detection on that 5 seconds:

In [ ]:
#@title
!rm -rf video.mp4
# cut the first 5 seconds
!ffmpeg -y -loglevel info -i "full_video.mp4" -t 5 video.mp4
# detect poses on the these 5 seconds
!rm -f openpose.avi
!cd openpose && ./build/examples/openpose/openpose.bin --video ../video.mp4 --write_json ./output/ --display 0 --part-candidates --write_video ../openpose.avi
# convert the result into MP4
!ffmpeg -y -loglevel info -i openpose.avi output.mp4

Display the video created by OpenPose:

In [ ]:
show_local_mp4_video('output.mp4', width=960, height=720)